In [1]:
from ultralytics import YOLO

In [2]:
model = YOLO("best.pt")

In [ ]:
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox
from PIL import Image
from twilio.rest import Client
import threading
import cv2
import torch

account_sid = 'AC8faa1d1b34626cd229aa3848ee2ebd63'
auth_token = 'c75e51379c543a5c63f4b7718609f65f'

client = Client(account_sid, auth_token)
# Load your model

def upload_video():
    file_path = filedialog.askopenfilename(filetypes=[("Video files", "*.mp4;*.avi;*.mov;*.mkv")])
    if file_path:
        result_label.config(text="Processing video...", fg="blue")
        threading.Thread(target=process_and_notify, args=(file_path,)).start()

def process_and_notify(file_path):
    count =0
    cap = cv2.VideoCapture(file_path)
    if not cap.isOpened():
        root.after(0, messagebox.showerror, "Error", "Failed to open video file.")
        return


    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to image format
        image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        # Process the image with your model
        results = model(image)

        for result in results:
            boxes = list(result.boxes.shape)  # Boxes object for bounding box outputs
            print(boxes[0])
        
        if boxes[0]>0 and count<2:
            count = count+1
            message = client.messages.create(
            body='''############ Alert ############
    Please Check the Ware House facility in Zone-1 area. There is a Oil spill.''',
        from_='whatsapp:+14155238886',  # Twilio phone number
        to= 'whatsapp:+919016939348' # Recipient's phone number
        )
            root.after(0, messagebox.showinfo, "Notification", f"Spill detected! Message sent with SID: {message.sid}")
        else:
            root.after(0, messagebox.showinfo, "Result", "No spill detected.")
    cap.release()

def update_ui(spill_detected):
    if spill_detected:
        result_label.config(text='''############ Alert ############
Please Check the Ware House facility in Zone-1 area. There is an Oil spill.''', fg="red")
    #else:
     #   result_label.config(text='No spill Detected', fg="green")

# Create the main window
root = tk.Tk()
root.title("Spill Detection")
root.geometry("400x400")
root.configure(bg="#f0f0f0")

# Create a frame to center align all elements
frame = tk.Frame(root, bg="#f0f0f0")
frame.place(relx=0.5, rely=0.5, anchor="center")

# Add a title label
title_label = tk.Label(frame, text="Spill Detection System", bg="#f0f0f0", font=("Helvetica", 20, "bold"))
title_label.pack(pady=10)

# Add an instruction label
instruction_label = tk.Label(frame, text="Upload a video to check for any spills. The system will notify if a spill is detected.", 
                             bg="#f0f0f0", font=("Helvetica", 12), wraplength=350)
instruction_label.pack(pady=10)

# Create a button to upload a video
upload_button = tk.Button(frame, text="Upload Video", command=upload_video, bg="#4CAF50", fg="white", font=("Helvetica", 15, "bold"))
upload_button.pack(pady=20)

# Create a label to show the result
result_label = tk.Label(frame, text="Result: ", bg="#f0f0f0", font=("Helvetica", 17))
result_label.pack(pady=20)

# Add a footer label
footer_label = tk.Label(frame, text=" Project by Team Nirmaan ", bg="#f0f0f0", font=("Helvetica", 10, "italic"))
footer_label.pack(pady=10)

# Run the Tkinter event loop
root.mainloop()